# 2 使用 Python 呼叫 API

OpenAI 官方提供有 openai 套件, 可以簡化直接使用 requests 模組的複雜度。

## 2-1 使用官方 openai 套件

### 安裝與使用 openai **套件**

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
import openai
openai.api_key = "你的金鑰"

In [ ]:
reply = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    # model = "gpt-4",
    messages = [
        {"role":"user", "content": "你好"}
    ]
)

檢視傳回物件

In [ ]:
print(reply)

In [ ]:
print(reply["choices"][0]["message"]["content"])

### 傳遞多筆訊息

In [ ]:
reply = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = [
        {"role":"system", "content":"你是條住在深海、只會台灣中文的魚"},
        {"role":"user", "content": "你住的地方很亮嗎？"}
    ]
)

In [ ]:
print(reply["choices"][0]["message"]["content"])

### 設定與隱藏金鑰的方法

在 Colab 中設定環境變數的方法, 不過這個方法會把金鑰留在 Colab 筆記本中, 我們不採用這個方法：

In [ ]:
%env OPENAI_API_KEY=你的金鑰

**使用環境變數隱藏金鑰：dotenv 模組**

以下採用 python-dotenv 套件, 可以從 .env 檔中讀取並設定環境變數, 請依照以下步驟準備好設定環境變數的 .env 檔。



**方法一：在本機建立環境變數隱藏檔**

使用記事本或是任何文字編輯軟體建立 .env 檔後再拖曳到 Colab 的檔案窗格上傳到 Colab 中。

**方法二：在 Colab 建立環境變數隱藏檔**

1. 請先建立 env 檔案
1. 在 env 檔案中依照 OPENAI_API_KEY=你的金鑰 這樣的格式記錄環境變數
1. 將 env 檔案重新命名為 .env 檔

之所以要先建立 env 檔再重新命名為 .env, 是因為 Colab 不允許編輯 . 開頭的隱藏檔。

使用 dotenv 依檔案內容建立環境變數

In [ ]:
!pip install python-dotenv

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import os
os.getenv("OPENAI_API_KEY")

程式中就可以不用出現金鑰了

## 2-2 認識 token

### 使用 tiktoken 套件計算精確 token 數

In [ ]:
!pip install tiktoken
import tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.4 MB/s eta 0:00:00


In [ ]:
encoder = tiktoken.encoding_for_model('gpt-3.5-turbo')
print(encoder.name)
encoder = tiktoken.encoding_for_model('gpt-4')
print(encoder.name)

In [ ]:
tokens = encoder.encode("你好")
print(tokens)

In [ ]:
print(encoder.decode(tokens))

### token 切割視覺化工具

In [ ]:
!pip install ansicolors
from colors import color

In [ ]:
print(color('淡橙色', bg='#FFB347'))
print(color('桃色', bg='#FFCC99'))
print(color('淡黃色', bg='#FFFF66'))
print(color('茶綠色', bg='#64fa64'))
print(color('天青色', bg='#CCFFCC'))
print(color('薊色', bg='#CB99C9'))
print(color('淺粉紅色', bg='#FFB6C1'))
print(color('粉翠珠色', bg='#FFCCDD'))
print(color('薰衣草藍', bg='#CCCCFF'))

淡橙色
桃色
淡黃色
茶綠色
天青色
薊色
淺粉紅色
粉翠珠色
薰衣草藍


In [ ]:
def tokenizer(msg, model="gpt-3.5-turbo", show=False):
    encoder = tiktoken.encoding_for_model(model)
    color_pallet = (
        '#FFB347', # Pastel orange (淡橙色)
        '#FFCC99', # Peach (桃色)
        '#FFFF66', # Pastel yellow (淡黃色)
        '#64fa64', # Tea green (茶綠色)
        '#CCFFCC', # Aero blue (天青色)
        '#CB99C9', # Thistle (薊色)
        '#FFB6C1', # Light pink (淺粉紅色)
        '#FFCCDD', # Pink lace (粉翠珠色)
        '#CCCCFF'  # Lavender blue (薰衣草藍)
    )
    idx = 0
    tokens = encoder.encode(msg)
    if show:
        for token in tokens:
            print(color(encoder.decode([token]),
                        bg=color_pallet[idx],
                        fg='black'),
                end='')
            idx = (idx + 1) % len(color_pallet)
        print()
    return tokens

In [ ]:
tokenizer('Alphabet Inc is Google\'s parent company.',
          show=True)

Alphabet Inc is Google's parent company.


[2149, 18992, 4953, 374, 5195, 596, 2748, 2883, 13]

In [ ]:
tokenizer('你好？', show=True)

### ChatML 標記語言

In [ ]:
print(tokenizer("user"))
print(tokenizer("assistant"))
print(tokenizer("system"))
print(tokenizer("\n"))

計算 message 總 tokens 數

In [ ]:
def tokens_in_messages(messages, model="gpt-3.5-turbo",
                       show=False):
    totals = 0
    for message in messages:
        for k in message:
            if k == "content":
                totals += 4 # <|im_start|>user\n{內容}<|im_end|>
                totals += len(tokenizer(message[k],
                                        model,
                                        show))
    totals += 3 # <|im_start|>assistant<|message|>
    return totals

In [ ]:
print(tokens_in_messages([
        {"role":"user", "content": "你好"}
    ]))

## 2-3 使用 Python requests 模組呼叫 API

### 使用 Python requests 模組

Python 提供有 requests 模組, 可以快速發送 HTTP 請求。<br>
利用 requests 模組叫用 API。

In [ ]:
# 如果是要在本機上執行, 必須安裝 requests 套件
!pip install requests

In [ ]:
import requests
import os

response = requests.post(
    'https://api.openai.com/v1/chat/completions',
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {os.getenv("OPENAI_API_KEY")}'
    },
    json = {
        'model': 'gpt-3.5-turbo',
        "messages": [{"role": "user", "content": "你好"}]}
)

In [ ]:
reply = response.json()

In [ ]:
print(reply["choices"][0]["message"]["content"])

### 利用 curl 工具快速測試 API


In [ ]:
%%bash
curl -s https://api.openai.com/v1/chat/completions \
-H 'Content-Type: application/json' \
-H "Authorization: Bearer $OPENAI_API_KEY" \
-d '{
      "model": "gpt-3.5-turbo",
      "messages": [
        {
          "role": "user", "content": "你好"
        }
      ]
    }'